# Heros of Pymoli Code

In [1]:
#Import Dependencies
import pandas as pd
import csv
import numpy as np

#Load Data from CSV
file_to_load = "Resources/purchase_data.csv"

purchase_data_df = pd.read_csv(file_to_load)

In [2]:
purchase_data_df.head(6)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61


## Player Count

In [3]:
#Selecting Desired Columns For Source Dataframe (omitting repeat screen names)
number_of_players = purchase_data_df["SN"].nunique()

#Creating Dataframe
number_of_players_df = pd.DataFrame({"Total Players": [number_of_players]})
number_of_players_df

,Total Players
0,576


### Purchasing Analysis (Total)

In [4]:
#Selecting Desired Columns For Source Dataframe (ommitting repeat items)
number_of_items = len(purchase_data_df["Item Name"].unique())

#Selecting Column from Source Dataframe and computing mean price
average_item_price = purchase_data_df["Price"].mean()
 
#Finding number of total purchases
number_of_purchases = len(purchase_data_df)

#Finding sum of total purchases
total_revenue = purchase_data_df["Price"].sum()

#Creating Dataframe
purchase_summary_df = pd.DataFrame({"Number of Unique Items": [number_of_items],
                                   "Average Price": ["${0:,.2f}".format(average_item_price)],
                                   "Number of Purchases":[number_of_purchases],
                                   "Total Revenue": ["${0:,.2f}".format(total_revenue)]})
purchase_summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

In [5]:
#Selecting Desired Columns For Source Dataframe 
gender_df = purchase_data_df[["SN","Gender"]]

#Removing Repeat Players
unique_gender_df = gender_df.drop_duplicates()

#Counting Males and Females
players_gender_df = pd.DataFrame(unique_gender_df["Gender"].value_counts())

#Calculating Percentage of Males and Females Out of Total Players
players_gender_df["Percentage of Players"] = (players_gender_df["Gender"]/unique_gender_df["Gender"].count()*100)

#Rename Column
players_gender_df = players_gender_df.rename(columns={"Gender":"Total Count"})

#Formatting
players_gender_df["Percentage of Players"] = players_gender_df["Percentage of Players"].map("{:.2f}%".format)
players_gender_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

In [6]:
#Selecting Desired Columns For Source Dataframe
gender_purchases_df = purchase_data_df[["SN","Gender","Price"]]

#Creating New Dataframes
total_purchase_gender = gender_purchases_df.groupby("Gender")

total_purchase_gender_df = pd.DataFrame(total_purchase_gender["Price"].sum())

total_purchase_count_gender_df = pd.DataFrame(total_purchase_gender["Price"].count())

average_purchase_price_gender_df = pd.DataFrame(total_purchase_gender["Price"].mean())

#Creating New Dataframe
player_purchase = gender_purchases_df.groupby("SN")
total_purchase_per_player_df = pd.DataFrame(player_purchase["Price"].sum())

#Merging Dataframe
total_purchase_per_player_df = pd.merge(total_purchase_per_player_df,unique_gender_df, on="SN")

total_purchase_per_player_df = total_purchase_per_player_df.drop(columns="SN")

#Resetting Index
total_purchase_per_player_df = total_purchase_per_player_df.reset_index(drop=False)

#Creating New Dataframe
average_total_per_gender = total_purchase_per_player_df.groupby("Gender")
average_total_per_gender_df = pd.DataFrame(average_total_per_gender["Price"].mean())

#Renaming Column
average_total_per_gender_df = average_total_per_gender_df.rename(columns={"Price":"Avg Total Purchase per Person"})

#Resetting Indicies
average_total_per_gender_df = average_total_per_gender_df.reset_index(drop=False)
total_purchase_count_gender_df = total_purchase_count_gender_df.reset_index(drop=False)
average_purchase_price_gender_df = average_purchase_price_gender_df.reset_index(drop=False)

#Merging Dataframes with Gender
purchasing_analysis_df1 = pd.merge(total_purchase_count_gender_df, average_purchase_price_gender_df, on="Gender")
purchasing_analysis_df2 = pd.merge(total_purchase_gender_df, average_total_per_gender_df, on="Gender" )
purchasing_analysis_3 = pd.merge(purchasing_analysis_df1, purchasing_analysis_df2, on="Gender")

#Renaming Columns
purchasing_analysis_3 = purchasing_analysis_3.rename(columns={"Price_x":"Purchase Count", 
                                                             "Price_y": "Average Purchase Price",
                                                             "Price": "Total Purchase Value"})
#Formatting Output for decimals and dollar signs
purchasing_analysis_3["Average Purchase Price"] = purchasing_analysis_3["Average Purchase Price"].map("${0:,.2f}".format)
purchasing_analysis_3["Total Purchase Value"] = purchasing_analysis_3["Total Purchase Value"].map("${0:,.2f}".format)
purchasing_analysis_3["Avg Total Purchase per Person"] = purchasing_analysis_3["Avg Total Purchase per Person"].map("${0:,.2f}".format)
purchasing_analysis_3

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,"$1,967.64",$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [7]:
#Selecting Desired Columns For Source Dataframe
age_demo_df = purchase_data_df[["Age", "SN"]].drop_duplicates(keep='last')

#Counting Ages
player_age_df = pd.DataFrame(age_demo_df["Age"].value_counts())

#Reset Index and Rename Columns
player_age_df = player_age_df.reset_index(drop=False)
player_age_df = player_age_df.rename(columns={"index": "Age", "Age" : "Total Count"})

#Create Bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
age_groups = [" <10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]


player_age_df["Age Category"] = pd.DataFrame(pd.cut(player_age_df["Age"], bins, labels= age_groups, include_lowest=True))

#Creating Dataframe
player_age_group = player_age_df.groupby(["Age Category"])
player_age_group_df = pd.DataFrame(player_age_group["Total Count"].sum())

number_of_players = len(purchase_data_df["SN"].unique())

#Calculating Percentage of Players In Each Age Bin (and formatting)
player_age_group_df["Percentage of Players"] = ((player_age_group_df["Total Count"] / number_of_players) * 100).map("{:.2f}%".format)
player_age_group_df

,Total Count,Percentage of Players
Age Category,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [8]:
#Selecting Desired Columns from Source Dataframe
purchasing_analysis_age_df = purchase_data_df[["Age", "SN", "Price"]]

#Creating Dataframe and Counting Ages
purchase_count_df = pd.DataFrame(purchasing_analysis_age_df["Age"].value_counts())

#Adjusting Dataframe Index
purchase_count_df = purchase_count_df.reset_index(drop=False)

#Renameing Columns
purchase_count_df = purchase_count_df.rename(columns={"Age":"Purchase Count", "index":"Age"})

purchase_count_df["Age Category"] = pd.DataFrame(pd.cut(purchase_count_df["Age"], bins, labels= age_groups, include_lowest=True))

#Creating Dataframe
purchase_count_group = purchase_count_df.groupby(["Age Category"])
purchase_count_group_df = pd.DataFrame(purchase_count_group["Purchase Count"].sum())

average_purchase_price_df = purchasing_analysis_age_df[["Age", "Price"]]

average_purchase_price_df["Age Category"] = pd.DataFrame(pd.cut(average_purchase_price_df["Age"], bins, labels= age_groups, include_lowest=True))

#Creating Dataframe and Calculating Mean Price per Age Category
average_purchase_price_group = average_purchase_price_df.groupby(["Age Category"])
average_purchase_price_group_df = pd.DataFrame(average_purchase_price_group["Price"].mean())

#Creating Dataframe and Calculating Total Purchase Value
total_purhcase_value_group = purchasing_analysis_age_df.groupby(["Age"])
total_purchase_value_group_df = pd.DataFrame(total_purhcase_value_group["Price"].sum())

#Adjusting Index
total_purchase_value_group_df = total_purchase_value_group_df.reset_index(drop=False)

total_purchase_value_group_df["Age Category"] = pd.DataFrame(pd.cut(total_purchase_value_group_df["Age"], bins, labels= age_groups, include_lowest=True))

#Creating Dataframe
total_purchase_value_group2 = total_purchase_value_group_df.groupby(["Age Category"])
total_purchase_value_group2_df = pd.DataFrame(total_purchase_value_group2["Price"].sum())

#Creating Dataframe
average_purchase_total_per_person_group = purchasing_analysis_age_df.groupby(["SN"])
average_purchase_total_per_person_group_df = pd.DataFrame(average_purchase_total_per_person_group["Price"].sum())

average_purchase_total_per_person_group_df = average_purchase_total_per_person_group_df.reset_index(drop=False)

##Selecting Desired Columns from Source Dataframe for new Dataframe
player_age_df = purchase_data_df[["SN", "Age"]]

#To Avoid Counting Repeat Screen Names
player_age_df = player_age_df.drop_duplicates(keep='last')

#Merge Dataframes by Screen Name
merged_df = pd.merge(average_purchase_total_per_person_group_df, player_age_df, on = "SN")

merged_df["Age Category"] = pd.DataFrame(pd.cut(merged_df["Age"], bins, labels= age_groups, include_lowest=True))

#Creating New Dataframe
merged_group = merged_df.groupby(["Age Category"])
merged_group_df = pd.DataFrame(merged_group["Price"].mean())

#Merging Dataframes for Final Desired Dataframe
purchase_analysis_merge1 = pd.merge(purchase_count_group_df, average_purchase_price_group_df, on = "Age Category")

purchase_analysis_merge2 = pd.merge(total_purchase_value_group2_df, merged_group_df, on = "Age Category")

final_merge = pd.merge(purchase_analysis_merge1, purchase_analysis_merge2, on = "Age Category" ) 

final_merge = final_merge.rename(columns = {"Price_x" : "Total Purchase Value", "Price_y": "Average Total Purchase per Person", "Price": "Average Purchase Price"})

#Formating Final Dataframe
final_merge["Average Purchase Price"] = final_merge["Average Purchase Price"].map("${:,.2f}".format)
final_merge["Total Purchase Value"] = final_merge["Total Purchase Value"].map("${:,.2f}".format)
final_merge["Average Total Purchase per Person"] = final_merge["Average Total Purchase per Person"].map("${:,.2f}".format)
final_merge

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Category,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [9]:
#Selecting Desired Columns From Source Dataframe
top_spenders_df = purchase_data_df[["SN", "Price"]]

#Creating Dataframe and Calulating Sum
top_spender_players_group = top_spenders_df.groupby(["SN"])
top_spender_players_group_total_df = pd.DataFrame(top_spender_players_group["Price"].sum() )

#New Dataframe for Average
top_spender_players_group_avg_df = pd.DataFrame(top_spender_players_group["Price"].mean() )

#New Dataframe
top_spender_players_group_count_df = pd.DataFrame(top_spender_players_group["Price"].count())

#Merging Dataframes by Screen Name
spender_merge1 = pd.merge(top_spender_players_group_count_df, top_spender_players_group_avg_df, on = "SN")

#Merging by Screen Name Again
spender_merge2 = pd.merge(spender_merge1, top_spender_players_group_total_df, on = "SN" )

#Sorting from Greatest to Least
spender_merge2 = spender_merge2.sort_values("Price" ,ascending=False)

#To Identify top 5 Spenders
spenders_summary_df = spender_merge2.head()

#Renameing Columns
spenders_summary_df = spenders_summary_df.rename(columns ={"Price_x" : "Purchase Count", "Price_y": "Average Purchase Price", "Price": "Total Purchase Value"})

#Formatting Final Dataframe
spenders_summary_df["Average Purchase Price"] = spenders_summary_df["Average Purchase Price"].map("${:,.2f}".format)
spenders_summary_df["Total Purchase Value"] = spenders_summary_df["Total Purchase Value"].map("${:,.2f}".format)
spenders_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [10]:
#Selecting Desired Columns From Source Dataframe
most_popular_df = purchase_data_df[["Item ID", "Item Name", "Price"]]

#Creating New Dataframe
most_popular_group = most_popular_df.groupby(["Item Name"])
most_popular_group_df = pd.DataFrame(most_popular_group["Item Name"].count())

#Renaming Columns
most_popular_group_df = most_popular_group_df.rename(columns= {"Item Name": "Purchase Count"})

#Selecting Desired Columns From Previous Dataframe
most_popular_price_df = most_popular_df[["Item ID","Price"]]

#
most_popular_price_df = most_popular_price_df.drop_duplicates(keep='last')

#Selecting Desired Columns From Previous Dataframe
most_popular_total_value = most_popular_df[["Item ID","Price"]]

#Creating New Dataframe
most_popular_total_value_group = most_popular_total_value.groupby("Item ID")
most_popular_total_value_group_df = pd.DataFrame(most_popular_total_value_group["Price"].sum())

#
item_names_df = most_popular_df[["Item ID", "Item Name"]].drop_duplicates(keep='last')

#Special case
most_popular_price_df.loc[most_popular_price_df["Item ID"]==92]

#Creating New Dataframe
average_duplicate_items_group = most_popular_price_df.groupby(["Item ID"])
average_duplicate_items_group_df = pd.DataFrame(average_duplicate_items_group["Price"].mean())

#Merging Dataframes by Item Name
most_popular_summary_merge1 = pd.merge(most_popular_group_df, item_names_df, on = "Item Name")

#Merging Dataframes by Item ID
most_popular_summary_merge2 = pd.merge(average_duplicate_items_group_df, most_popular_total_value_group_df, on = "Item ID")

#Another Merge by Item ID
most_popular_summary_merge3 = pd.merge(most_popular_summary_merge1, most_popular_summary_merge2, on = "Item ID")

#Reseting Index
most_popular_summary_merge3 = most_popular_summary_merge3.set_index("Item ID","Item Name")

#Renaming Columns
most_popular_summary_merge3 = most_popular_summary_merge3.rename(columns= {"Price_x": "Item Price", "Price_y":"Total Purchase Value"})

#Sorting From Greatest to Least
most_popular_summary_merge3 = most_popular_summary_merge3.sort_values("Purchase Count" ,ascending=False)

#Selecting Top 5 Items
most_popular_summary_head = most_popular_summary_merge3.head()

#Formatting
most_popular_summary_head["Item Price"] = most_popular_summary_head["Item Price"].map("${:,.2f}".format)

most_popular_summary_head["Total Purchase Value"] = most_popular_summary_head["Total Purchase Value"].map("${:,.2f}".format)
most_popular_summary_head

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.54,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
132,Persuasion,9,$3.26,$28.99
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

In [11]:
#Sorting from Greatest to Least
most_popular_summary_most_profitable = most_popular_summary_merge3.sort_values("Total Purchase Value" ,ascending=False)

#Selecting Top 5 Items
most_popular_summary_most_profitable_head = most_popular_summary_most_profitable.head()

#Formatting
most_popular_summary_most_profitable_head["Item Price"] = most_popular_summary_most_profitable_head["Item Price"].map("${:,.2f}".format)

most_popular_summary_most_profitable_head["Total Purchase Value"] = most_popular_summary_most_profitable_head["Total Purchase Value"].map("${:,.2f}".format)

most_popular_summary_most_profitable_head

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.54,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
